In [32]:
import matplotlib.pyplot as plt

import pandas as pd
import requests
import json
import csv

In [65]:


#Get and clean premier league data
bs_response = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')

bs_data = json.loads(bs_response.text)

#assign 
events = bs_data['events']
players = bs_data['elements']
teams = bs_data['teams'] 


In [57]:
#convert positions to dataframe and extract relevant columns to merge later
positions_df = pd.DataFrame(positions)
positions_df = positions_df[['id', 'singular_name']]
positions_df.columns = ['element_type', 'position_name']
positions_df

,element_type,position_name
0,1,Goalkeeper
1,2,Defender
2,3,Midfielder
3,4,Forward


In [112]:
#Make dataframe with name, id and position id
players_df = pd.DataFrame(players)

players_df = players_df[['id', 'first_name', 'second_name', 'element_type', 'team']]
players_df.columns = ['player_id', 'first_name', 'second_name', 'element_type', 'team_id']
players_df['name'] = players_df['first_name'] + ' ' +  players_df['second_name']
players_df.drop(['first_name', 'second_name'], axis=1, inplace=True)
players_df.head()


,player_id,element_type,team_id,name
0,1,3,1,Mesut Özil
1,2,2,1,Sokratis Papastathopoulos
2,3,2,1,David Luiz Moreira Marinho
3,4,3,1,Pierre-Emerick Aubameyang
4,5,2,1,Cédric Soares


In [115]:
#merge positions with player position ids

player_pos_df = players_df.merge(positions_df, left_on='element_type', right_on='element_type', how='inner')
player_pos_df.sort_values(by=['player_id'], inplace=True)
player_pos_df = player_pos_df.set_index('player_id')
player_pos_df = player_pos_df.drop('element_type', axis=1)
player_pos_df = player_pos_df[['name', 'position_name', 'team_id']]
player_pos_df.head()


,name,position_name,team_id
player_id,,,
1,Mesut Özil,Midfielder,1
2,Sokratis Papastathopoulos,Defender,1
3,David Luiz Moreira Marinho,Defender,1
4,Pierre-Emerick Aubameyang,Midfielder,1
5,Cédric Soares,Defender,1


In [107]:


#create list of player ids and teams to join with stats

team_list = {}
for team in teams:
    team_list[team['id']] = team['name']


team_df = pd.DataFrame.from_dict(team_list, orient='index', columns=['team_name'])
#print(team_df.head()) # team_id: team_name

player_on_team = {}
for player in players:
    player_on_team[player['id']] = player['team']
player_on_team_df = pd.DataFrame.from_dict(player_on_team, orient='index', columns=['team_id'])

#join team list and player on team to add to csv file
players_teams_df = player_on_team_df.join(team_df, on='team_id')
players_teams_df = players_teams_df.reset_index()
players_teams_df.head()


,index,team_id,team_name
0,1,1,Arsenal
1,2,1,Arsenal
2,3,1,Arsenal
3,4,1,Arsenal
4,5,1,Arsenal


In [120]:
#make dataframe with teams and their ids to merge with players_df
teams_df = pd.DataFrame(teams)
teams_df = teams_df[['id', 'name']]
teams_df.columns = ['team_id', 'team_name']
teams_df.head()

players_df = player_pos_df.merge(teams_df, left_on='team_id', right_on='team_id')
players_df.head()

,name,position_name,team_id,team_name
0,Mesut Özil,Midfielder,1,Arsenal
1,Sokratis Papastathopoulos,Defender,1,Arsenal
2,David Luiz Moreira Marinho,Defender,1,Arsenal
3,Pierre-Emerick Aubameyang,Midfielder,1,Arsenal
4,Cédric Soares,Defender,1,Arsenal


In [37]:
#Generate links for each player
# url = 'https://fantasy.premierleague.com/api/element-summary/' + str(player_dict[player]) + '/'
player_urls = []

for player in players:
    player_urls.append('https://fantasy.premierleague.com/api/element-summary/' + str(player['id']) + '/')


In [40]:
#Loop through player URLS and add the results to a dataframe and save it to a csv fil
players_df = None
 
# create the csv writer object
csv_writer = csv.writer(data_file)

'''
#uncomment to run url calls
# now we will open a file for writing
data_file = open('player_stats.csv', 'w')

count = 0
for url in player_urls:
    response = requests.get(url)
    player_data =  response.json() #creates python dictionary from json response
    
    for player in player_data['history']:
        if count == 0:
            header = player.keys()
            csv_writer.writerow(header)
            count+=1
        csv_writer.writerow(player.values())

data_file.close()
'''

In [48]:
#data wrangling to combine team names and player names to csv file
#read csv file into pandas dataframe
#df = pd.read_csv('player_stats.csv', header=0, index_col=0)
#display(df.head())
#display(players_teams_df.head())
#display(player_id_df.head())

df_with_teams = df.merge(players_teams_df, left_on='element', right_on='index', how='inner')
df_with_teams.head() 

df_with_teams = df_with_teams.merge(player_id_df, left_on='index', right_on='index', how='inner')
df_with_teams




,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,...,ict_index,value,transfers_balance,selected,transfers_in,transfers_out,index,team_id,team_name,name
0,2,8,0,False,2020-09-12T11:30:00Z,0,3,1,0,0,...,0.0,70,0,76656,0,0,1,1,Arsenal,Mesut Özil
1,9,19,0,True,2020-09-19T19:00:00Z,2,1,2,0,0,...,0.0,69,-16828,68335,995,17823,1,1,Arsenal,Mesut Özil
2,23,11,0,False,2020-09-28T19:00:00Z,3,1,3,0,0,...,0.0,69,-11451,59793,675,12126,1,1,Arsenal,Mesut Özil
3,29,15,0,True,2020-10-04T13:00:00Z,2,1,4,0,0,...,0.0,68,-5324,56403,647,5971,1,1,Arsenal,Mesut Özil
4,44,12,0,False,2020-10-17T16:30:00Z,1,0,5,0,0,...,0.0,68,-4224,53689,616,4840,1,1,Arsenal,Mesut Özil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21300,255,14,2,False,2021-02-27T20:00:00Z,1,1,26,82,0,...,1.7,70,-1925,20524,4746,6671,642,20,Wolves,Willian José Da Silva
21301,284,12,0,False,2021-03-02T20:00:00Z,4,1,26,0,0,...,0.0,70,-1925,20524,4746,6671,642,20,Wolves,Willian José Da Silva
21302,259,2,2,False,2021-03-06T17:30:00Z,0,0,27,77,0,...,2.4,70,-4078,16772,1448,5526,642,20,Wolves,Willian José Da Silva
21303,278,11,2,True,2021-03-15T20:00:00Z,0,1,28,69,0,...,1.0,69,-2932,14065,1045,3977,642,20,Wolves,Willian José Da Silva


In [19]:
#adds gameweek as new column

'''
gameweeks = set()
kickoff_time = df['kickoff_time']
for gameweek in kickoff_time:
    g = gameweek.split('T')[0]
    gameweeks.add(g)

gameweeks = list(gameweeks)
gameweeks = sorted(gameweeks, reverse=True)


new = df['kickoff_time'].str.split('T', n = 1, expand= True)

df['gameweek'] = new[0]
'''


In [27]:
display(df.head())
player_pos_df.head()



,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,transfers_out,index,team_id,team_name,name,gameweek,element_type_x,position_name_x,element_type_y,position_name_y
0,1,2,8,0,False,2020-09-12T11:30:00Z,0.0,3.0,1,0,...,0,1,1,Arsenal,Mesut Özil,2020-09-12,3,Midfielder,3,Midfielder
1,1,9,19,0,True,2020-09-19T19:00:00Z,2.0,1.0,2,0,...,17823,1,1,Arsenal,Mesut Özil,2020-09-19,3,Midfielder,3,Midfielder
2,1,23,11,0,False,2020-09-28T19:00:00Z,3.0,1.0,3,0,...,12126,1,1,Arsenal,Mesut Özil,2020-09-28,3,Midfielder,3,Midfielder
3,1,29,15,0,True,2020-10-04T13:00:00Z,2.0,1.0,4,0,...,5971,1,1,Arsenal,Mesut Özil,2020-10-04,3,Midfielder,3,Midfielder
4,1,44,12,0,False,2020-10-17T16:30:00Z,1.0,0.0,5,0,...,4840,1,1,Arsenal,Mesut Özil,2020-10-17,3,Midfielder,3,Midfielder


,element_type,position_name
player_id,,
1,3,Midfielder
4,3,Midfielder
9,3,Midfielder
18,3,Midfielder
19,3,Midfielder


,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,...,value,transfers_balance,selected,transfers_in,transfers_out,index,team_id,team_name,name,gameweek
player_id,,,,,,,,,,,,,,,,,,,,,
0,2,8,0,False,2020-09-12T11:30:00Z,0.0,3.0,1,0,0,...,70,0,76656,0,0,1,1,Arsenal,Mesut Özil,2020-09-12
1,9,19,0,True,2020-09-19T19:00:00Z,2.0,1.0,2,0,0,...,69,-16828,68335,995,17823,1,1,Arsenal,Mesut Özil,2020-09-19
2,23,11,0,False,2020-09-28T19:00:00Z,3.0,1.0,3,0,0,...,69,-11451,59793,675,12126,1,1,Arsenal,Mesut Özil,2020-09-28
3,29,15,0,True,2020-10-04T13:00:00Z,2.0,1.0,4,0,0,...,68,-5324,56403,647,5971,1,1,Arsenal,Mesut Özil,2020-10-04
4,44,12,0,False,2020-10-17T16:30:00Z,1.0,0.0,5,0,0,...,68,-4224,53689,616,4840,1,1,Arsenal,Mesut Özil,2020-10-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,89,20,0,True,2020-11-29T19:15:00Z,1.0,2.0,10,0,0,...,50,-15129,51099,682,15811,567,1,Arsenal,Thomas Partey,2020-11-29
996,106,17,1,False,2020-12-06T16:30:00Z,2.0,0.0,11,45,0,...,50,-10758,40546,525,11283,567,1,Arsenal,Thomas Partey,2020-12-06
997,109,4,0,True,2020-12-13T19:15:00Z,0.0,1.0,12,0,0,...,50,-5346,35876,702,6048,567,1,Arsenal,Thomas Partey,2020-12-13


## add position to csv

In [21]:
test = df.merge(player_pos_df, how='inner', left_on='element', right_on='player_id')
#df.to_csv('player_stats.csv')


#test.to_csv('player_stats_w_pos.csv', index=True)
df = pd.read_csv('player_stats_w_pos.csv', header=0, index_col=0)


In [26]:
test.head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,transfers_out,index,team_id,team_name,name,gameweek,element_type_x,position_name_x,element_type_y,position_name_y
0,1,2,8,0,False,2020-09-12T11:30:00Z,0.0,3.0,1,0,...,0,1,1,Arsenal,Mesut Özil,2020-09-12,3,Midfielder,3,Midfielder
1,1,9,19,0,True,2020-09-19T19:00:00Z,2.0,1.0,2,0,...,17823,1,1,Arsenal,Mesut Özil,2020-09-19,3,Midfielder,3,Midfielder
2,1,23,11,0,False,2020-09-28T19:00:00Z,3.0,1.0,3,0,...,12126,1,1,Arsenal,Mesut Özil,2020-09-28,3,Midfielder,3,Midfielder
3,1,29,15,0,True,2020-10-04T13:00:00Z,2.0,1.0,4,0,...,5971,1,1,Arsenal,Mesut Özil,2020-10-04,3,Midfielder,3,Midfielder
4,1,44,12,0,False,2020-10-17T16:30:00Z,1.0,0.0,5,0,...,4840,1,1,Arsenal,Mesut Özil,2020-10-17,3,Midfielder,3,Midfielder


In [24]:
sm_df = df[['element', 'name', 'team_name', 'position_name','round', 'total_points']]
display(sm_df.head())
sm_df.to_csv('player_stats_sm.csv', index=True)

KeyError: "['position_name'] not in index"